In [1]:
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


from plyfile import PlyData, PlyElement
import numpy as np


In [2]:
init_notebook_mode(connected=True)

### Gather the xyz points from the ply file

In [3]:
plydata = PlyData.read('ply/cow2.ply')

nr_points=plydata.elements[0].count
nr_faces=plydata.elements[1].count

# unpack vertices to array
vertices_pt_xyz =np.array([np.array(plydata['vertex'][k]) for k in range(nr_points)])

vertices_pt_xyz=np.array(map(list, vertices_pt_xyz))
vertices_pt_xyz[:,2] -= 5 # down the z axis
x,y,z=zip(*vertices_pt_xyz)

# unpack faces to array
faces_pt_3v = np.array([np.array(plydata['face'][k][0]) for k in range(nr_faces)])
faces_pt_3v[0]

array([0, 1, 2], dtype=int32)

### Plot!

In [4]:

cowScat =Scatter3d(x=x,
                y=y,
                z=z,
                mode='markers',
                marker=dict(
                    size=2,
                    color=z,                # set color to an array/list of desired values
                    colorscale='Red',   # choose a colorscale
                    opacity=0.8)
               )

        
axis=dict(showbackground=False,
          showline=True,  
          zeroline=True,
          showgrid=True,
          showticklabels=True,
          title='' 
          )

layout = Layout(title="PLY", 
                width=800,
                height=800,
                showlegend=False,
                scene=Scene(aspectmode='manual',
                            aspectratio=dict(x=1, y=1, z=1),
                            xaxis=dict(range=[-10, 10], title='x Axis'),
                            yaxis=dict(range=[-10, 10], title='y Axis'),
                            zaxis=dict(range=[-20,0], title='z Axis'),
                            camera = dict(
                                up=dict(x=1, y=0, z=0),
                                center=dict(x=0, y=0, z=0),
                                eye=dict(x=0, y=0, z=2)
                            ),

                           ),
                margin=Margin(t=100),
                hovermode='closest',
                
                )

data=Data([cowScat])
fig=Figure(data=data, layout=layout)
iplot(fig)

NameError: name 'xMark' is not defined

### Make a projection matrix

In [5]:
fovy = 90.0 * (np.pi/180)
aspect = 1.0;
fovx = fovy * aspect;
nearDist = 0.1;
farDist = 10.0;

yScale = 1.0 / np.tan(fovy / 2);
xScale = yScale/aspect;
nearmfar = nearDist - farDist;

In [6]:
projMat_4x4 = np.zeros([4,4]);
projMat_4x4[0,0] = xScale
projMat_4x4[1,1] = yScale
projMat_4x4[2,2] = (farDist+nearDist) / (nearDist-farDist);
projMat_4x4[2,3] = -1.0;
projMat_4x4[3,2] = 2*(nearDist * farDist) / (nearDist-farDist)
projMat_4x4[3,3] = 1

projMat_4x4

array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -1.02020202, -1.        ],
       [ 0.        ,  0.        , -0.2020202 ,  1.        ]])

### Add the W to XYZ (homogenous coordinates)

In [7]:
homVertices_vIdx_xyzw = [np.hstack([list(plydata['vertex'][k]),1.]) for k in range(nr_points)]

In [8]:
homVertices_vIdx_xyzw[0:5]

[array([ 0.60553801,  0.18312199, -0.472278  ,  1.        ]),
 array([ 0.64922303,  0.12970001, -0.49487501,  1.        ]),
 array([ 0.60108203,  0.105512  , -0.53334302,  1.        ]),
 array([ 0.69124502,  0.0569483 , -0.52476197,  1.        ]),
 array([ 0.652035  ,  0.0379582 , -0.54233199,  1.        ])]

### Project using matrix multiplication

In [1]:
def projectVertex(xyzw,projMatIn_4x4):
    dataOutXYZW = np.matmul( projMatIn_4x4,np.transpose(xyzw))
    #dataOutXYZ = dataOutXYZW[0:3] / dataOutXYZW[3]
    dataOutXYZ = dataOutXYZW[0:3]
    return dataOutXYZ
    
projVertices_vIdx_xyzw = np.apply_along_axis(projectVertex,1,homVertices_vIdx_xyzw,projMat_4x4)

NameError: name 'np' is not defined

In [11]:
x2,y2,z2=zip(*projVertices_vIdx_xyzw[:,0:3])

### Plot!

In [14]:
from trimesh import *

cowSurf = plotly_trisurf(x,y,z, faces_pt_3v, plot_edges=None)

projSurf = gd_profSurf(x2,y2,z,-1, faces_pt_3v, plot_edges=None)

        
axis=dict(showbackground=False,
          showline=True,  
          zeroline=True,
          showgrid=True,
          showticklabels=True,
          title='' 
          )

layout = Layout(title="PLY", 
                width=800,
                height=800,
                showlegend=False,
                scene=Scene(aspectmode='manual',
                            aspectratio=dict(x=1, y=1, z=1),
                            xaxis=dict(range=[-10, 10], title='x Axis'),
                            yaxis=dict(range=[-10, 10], title='y Axis'),
                            zaxis=dict(range=[-20,0], title='z Axis'),
                            camera = dict(
                                up=dict(x=1, y=0, z=0),
                                center=dict(x=0, y=0, z=0),
                                eye=dict(x=0, y=0, z=2)
                            ),

                           ),
                margin=Margin(t=100),
                hovermode='closest',
                
                )
data = [cowSurf[0],projSurf[0]]
fig=Figure(data=data, layout=layout)
iplot(fig)


# Calc distance to each line

### Get all the lines in the polygon from its faces

In [15]:
def returnLines(face_3pt,vertices2D_pt_xy):
    A = vertices2D_pt_xy[face_3pt[[0,1]]]
    B = vertices2D_pt_xy[face_3pt[[0,2]]]
    C = vertices2D_pt_xy[face_3pt[[1,2]]]
    return A,B,C
    #return [A,B],[B,C],[A,C]

vertices2D_pt_xy = np.array(zip(x2,y2))
lines_lIdx_ptAB_xy = np.vstack([returnLines(face,vertices2D_pt_xy) for face in faces_pt_3v])

### Find closest point to gaze point

In [16]:
gazePt_xy = [5,3.4]

def dist(line_2pt_xy, C):
    """Calculate the distance of point C to line segment spanned by points A, B.

    """

    a = np.asarray(line_2pt_xy[0,:])
    b = np.asarray(line_2pt_xy[1,:])
    c = np.asarray(C)

    n, v = b - a, c - a
    t = max(0, min(np.dot(v, n)/np.dot(n, n), 1))
    
    #return [(a + t*n) , np.linalg.norm(c - (a + t*n)) ]
    return (np.linalg.norm(c - (a + t*n)) , (a + t*n) )


dist_line,closestPt_ln_xy = zip(*[dist(line,gazePt_xy) for line in lines_lIdx_ptAB_xy])
minIdx = np.argmin(dist_line)

minDist = dist_line[minIdx]
closestPt_xy = closestPt_ln_xy[minIdx]

# Plot!

In [ ]:
from trimesh import *


gazePt_xy = [1,5]

dist_line,closestPt_ln_xy = zip(*[dist(line,gazePt_xy) for line in lines_lIdx_ptAB_xy])
minIdx = np.argmin(dist_line)

minDist = dist_line[minIdx]
closestPt_xy = closestPt_ln_xy[minIdx]

gX,gY = zip(gazePt_xy,closestPt_xy)
gazeVec =Scatter3d(x=gX,
                y=gY,
                z=[-1,-1],
                mode='markers+lines',
                marker=dict(
                    size=3,
                    color=z,                # set color to an array/list of desired values
                    colorscale='Red',   # choose a colorscale
                    opacity=0.8)
               )




cowSurf = plotly_trisurf(x,y,z, faces_pt_3v, plot_edges=None)

projSurf = gd_profSurf(x2,y2,z,-1, faces_pt_3v, plot_edges=None)

        
axis=dict(showbackground=False,
          showline=True,  
          zeroline=True,
          showgrid=True,
          showticklabels=True,
          title='' 
          )

layout = Layout(width=800,
                height=800,
                showlegend=False,
                scene=Scene(aspectmode='manual',
                            aspectratio=dict(x=1, y=1, z=1),
                            xaxis=dict(range=[-10, 10], title='x Axis'),
                            yaxis=dict(range=[-10, 10], title='y Axis'),
                            zaxis=dict(range=[-10,0], title='z Axis'),
                            camera = dict(
                                up=dict(x=1, y=0, z=0),
                                center=dict(x=0, y=0, z=0),
                                eye=dict(x=0, y=0, z=2)
                            ),

                           ),
                margin=Margin(t=100),
                hovermode='closest',
                
                )
data = [cowSurf[0],projSurf[0],gazeVec]
fig=Figure(data=data, layout=layout)
iplot(fig)

